In [1]:
import altair as alt
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
save=True

In [3]:
def makeDotPlot(df,saveName='DotPlot.png',title='Macrophage Exercise Score, Post v Pre',
                x='ROC-AUC',y='ID',color='Exercise Type',size='-Log10 P value',opac='Significant',rev=True,save=True):
    chart=alt.Chart(df).mark_circle().encode(x=alt.X(x,scale=alt.Scale(domain=[0, 1])),
                                             y=y,
                                             color=color,
                                             opacity=opac,
                                             size=alt.Size(size,
                                                           scale=alt.Scale(range=[100,500],
                                                                           reverse=rev))).properties(title=title)
    if save:
        chart.save(saveName)
    return chart

In [4]:
sedEx=pd.read_csv('./Files/Macrophages_Sed_Ex.txt',sep='\t')
prePost=pd.read_csv('./Files/Macrophages_Pre_Post.txt',sep='\t')


prePost.insert(3,'Exercise Type','Immediate')
sedEx.insert(3,'Exercise Type','Training')

df=pd.concat([prePost,sedEx])
df['-Log10 P value']=-np.log10(df['Pval'])
df['Significant']=df['Pval']<=0.05
df.index=df['ID']
if save:
    df.to_excel('./Files/SupplementaryTable1_Immediate_and_Training_Datasets_ROCAUC_Pvalues.xlsx',index=False)
data2=pd.read_csv('./Files/MacrophageExercise_DatasetsInfo.csv',index_col=0)

In [5]:
df

,ID,ROC-AUC,Pval,Exercise Type,-Log10 P value,Significant
ID,,,,,,
GSE198266,GSE198266,0.00,1.610000e-07,Immediate,6.793174,True
EMTAB1788,EMTAB1788,0.35,4.480000e-02,Immediate,1.348722,True
GSE72462,GSE72462,0.36,5.570000e-01,Immediate,0.254145,False
GSE140089,GSE140089,0.17,8.530000e-03,Immediate,2.069051,True
GSE53598,GSE53598,0.31,7.960000e-02,Immediate,1.099087,False
...,...,...,...,...,...,...
GSE28998,GSE28998,0.80,3.960000e-02,Training,1.402305,True
GSE9103,GSE9103,0.51,8.580000e-01,Training,0.066513,False
GSE58249,GSE58249,0.50,9.960000e-01,Training,0.001741,False


In [6]:
data2

,Number Samples,Species,Experiment Type,Tissues,Genders,Ages,Citation,Exercise Type
ID,,,,,,,,
E-MEXP-740,32,Homo sapiens,transcription profiling by array,{},{'Male': 32},{'69 to 73': 32},https://doi.org/10.1093/gerona/62.10.1088,Resistance/Strength
GSE104079,58,Mus musculus,Expression profiling by array,"{'Soleus': 27, 'Liver': 31}",{'Male': 58},{'6 month old': 58},https://doi.org/10.1096/fj.201701378RR,Aerobic/Endurance
GSE104999,24,Homo sapiens,Expression profiling by array,{'Vastus': 24},{'Male': 24},{},https://doi.org/10.1152/ajpregu.00452.2017,Immobilization
GSE109657,22,Homo sapiens,Expression profiling by array,{'Vastus': 22},{'Male': 22},{},https://doi.org/10.1038/s41598-018-35115-x,HITT
GSE110747,44,Mus musculus,Expression profiling by array,{'Liver': 44},{'Male': 44},{'11-12 weeks': 44},https://doi.org/10.3390/nu10050547,Aerobic/Endurance
...,...,...,...,...,...,...,...,...
GSE87748,20,Homo sapiens,Expression profiling by high throughput sequen...,{'Skeletal': 20},{},{},https://doi.org/10.3389/fendo.2016.00165,Resistance/Strength
GSE9103,40,Homo sapiens,Expression profiling by array,{},{},{},https://doi.org/10.2337/db08-0349,Aerobic/Endurance
GSE9405,21,Homo sapiens,Expression profiling by array,{},{},{},https://doi.org/10.1249/MSS.0b013e31818c6be9,Mixed


## Figure 4 A

In [10]:
resLst=list(data2[data2['Exercise Type']=='Resistance/Strength'].index)
saveName='./Files/Figs/MacrophageExerciseDotPlot_Resistance_Training_log10Pval.svg'
title='Resistance Exercise'
dfIn=df.loc[resLst]

chart=makeDotPlot(dfIn,saveName=saveName,title=title,rev=False,save=save)
chart

alt.Chart(...)

In [11]:
#endurLst=list(data2[data2['Exercise Type']=='Aerobic/Endurance'].index)
endurLst=['GSE104079','GSE111555','GSE117070','GSE11803','GSE120862',
          'GSE122671','GSE139258','GSE155271','GSE198266','GSE4252',
          'GSE1786','GSE178262','GSE179394','GSE221210','GSE28498',
          'GSE3606','GSE40551','GSE467','GSE51216','GSE83578']
saveName='./Files/Figs/MacrophageExerciseDotPlot_Endurance_Training_log10Pval.svg'
title='Endurance Exercise'
dfIn=df.loc[endurLst]

chart=makeDotPlot(dfIn,saveName=saveName,title=title,rev=False,save=save)
chart

alt.Chart(...)

## Figure 4 B 

In [19]:
bloodLst=['GSE111555', 'GSE122671', 'GSE28498','GSE34788','GSE46075','GSE5105','GSE68072']
saveName='./Files/Figs/MacrophageExerciseDotPlot_BloodSample_log10Pval.svg'
title='PBMC Blood'
dfIn=df.loc[bloodLst]

chart=makeDotPlot(dfIn,saveName=saveName,title=title,rev=False,save=save)
chart

alt.Chart(...)

In [18]:
df2Lst=['GSE104079','GSE117070','GSE144304','GSE9103','GSE199225','GSE99963']
saveName='./Files/Figs/MacrophageExerciseDotPlot_MuscleSample_log10Pval.svg'
title='Muscle Biopsy'
dfIn=df.loc[df2Lst]

chart=makeDotPlot(dfIn,saveName=saveName,title=title,rev=False,save=save)
chart

alt.Chart(...)

## Figure 4 C

In [20]:
humanLst=['GSE111555','GSE117070','GSE120862','GSE144304','GSE51216']
saveName='./Files/Figs/MacrophageExerciseDotPlot_Human_log10Pval.svg'
title='Human'
save=True
dfIn=df.loc[humanLst]

chart=makeDotPlot(dfIn,saveName=saveName,title=title,rev=False,save=save)
chart

alt.Chart(...)

In [21]:
mouseLst=['GSE104079','GSE198266','GSE11803','GSE178262','GSE467']

saveName='./Files/Figs/MacrophageExerciseDotPlot_Mouse_log10Pval.svg'
title='Mouse'
dfIn=df.loc[mouseLst]

chart=makeDotPlot(dfIn,saveName=saveName,title=title,rev=False,save=save)
chart

alt.Chart(...)

## Figure 4 D

In [22]:
rnaSeqLst=['GSE120862','GSE140089','GSE198266','GSE178262','GSE144304','GSE221210','GSE99963']
saveName='./Files/Figs/MacrophageExerciseDotPlot_RNASeq_log10Pval.svg'
title='RNA-Seq'
save=True
dfIn=df.loc[rnaSeqLst]

chart=makeDotPlot(dfIn,saveName=saveName,title=title,rev=False,save=save)
chart

alt.Chart(...)

In [23]:
arrayDf=['GSE104079','GSE117525','GSE122671','GSE16907','GSE28498','GSE44051','GSE9103']
saveName='./Files/Figs/MacrophageExerciseDotPlot_MicroArray_log10Pval.svg'
title='Microarray'
save=True
dfIn=df.loc[arrayDf]

chart=makeDotPlot(dfIn,saveName=saveName,title=title,rev=False,save=save)
chart

alt.Chart(...)